# ⚠ Warning

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gl/OpinionatedGeek%2Fmango-explorer/HEAD?filepath=AccountScout.ipynb) _🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

[🥭 Mango Markets](https://mango.markets/) support is available at: [Docs](https://docs.mango.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/mangomarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

# 🥭 AccountScout

If you want to run this code to check a user account, skip down to the **Running** section below and follow the instructions there.

(A 'user account' here is the root SOL account for a user - the one you have the private key for, and the one that owns sub-accounts.)


## Required Accounts

Mango Markets code expects some accounts to be present, and if they're not present some actions can fail.

From [Daffy on Discord](https://discord.com/channels/791995070613159966/820390560085835786/834024719958147073):

> You need an open orders account for each of the spot markets Mango. And you need a token account for each of the tokens.

This notebook (and the `AccountScout` class) can be used to check the required accounts are present and maybe in future set up all the required accounts.

(There's no reason not to write the code to fix problems and create any missing accounts. It just hasn't been done yet.)


In [ ]:
import logging
import typing

from solana.publickey import PublicKey

from BaseModel import Group, MarginAccount, OpenOrders
from Constants import SYSTEM_PROGRAM_ADDRESS
from Context import Context
from Wallet import Wallet


# ScoutReport class

The `ScoutReport` class is built up by the `AccountScout` to report errors, warnings and details pertaining to a user account.

In [ ]:
class ScoutReport:
    def __init__(self, address: PublicKey):
        self.address = address
        self.errors: typing.List[str] = []
        self.warnings: typing.List[str] = []
        self.details: typing.List[str] = []

    @property
    def has_errors(self) -> bool:
        return len(self.errors) > 0

    @property
    def has_warnings(self) -> bool:
        return len(self.warnings) > 0

    def add_error(self, error) -> None:
        self.errors += [error]

    def add_warning(self, warning) -> None:
        self.warnings += [warning]

    def add_detail(self, detail) -> None:
        self.details += [detail]

    def __str__(self) -> str:
        def _pad(text_list: typing.List[str]) -> str:
            if len(text_list) == 0:
                return "None"
            padding = "\n        "
            return padding.join(map(lambda text: text.replace("\n", padding), text_list))

        error_text = _pad(self.errors)
        warning_text = _pad(self.warnings)
        detail_text = _pad(self.details)
        if len(self.errors) > 0 or len(self.warnings) > 0:
            summary = f"Found {len(self.errors)} error(s) and {len(self.warnings)} warning(s)."
        else:
            summary = "No problems found"

        return f"""« ScoutReport [{self.address}]:
    Summary:
        {summary}

    Errors:
        {error_text}

    Warnings:
        {warning_text}

    Details:
        {detail_text}
»"""

    def __repr__(self) -> str:
        return f"{self}"


# AccountScout class

The `AccountScout` class aims to run various checks against a user account to make sure it is in a position to run the liquidator.

Passing all checks here with no errors will be a precondition on liquidator startup.

In [ ]:
class AccountScout:
    def __init__(self):
        pass

    def require_account_prepared_for_group(self, context: Context, group: Group, account_address: PublicKey) -> None:
        report = self.verify_account_prepared_for_group(context, group, account_address)
        if report.has_errors:
            raise Exception(f"Account '{account_address}' is not prepared for group '{group.address}':\n\n{report}")

    def verify_account_prepared_for_group(self, context: Context, group: Group, account_address: PublicKey) -> ScoutReport:
        report = ScoutReport(account_address)

        # First of all, the account must actually exist. If it doesn't, just return early.
        root_account = context.load_account(account_address)
        if root_account is None:
            report.add_error(f"Root account '{account_address}' does not exist.")
            return report

        # For this to be a user/wallet account, it must be owned by 11111111111111111111111111111111.
        if root_account.owner != SYSTEM_PROGRAM_ADDRESS:
            report.add_error(f"Account '{account_address}' is not a root user account.")
            return report

        # Must have token accounts for each of the tokens in the group's basket.
        for token in group.tokens:
            token_account  = context.fetch_token_balance(account_address, token.mint)
            if token_account is None:
                report.add_error(f"Account '{account_address}' has no account for token '{token.name}', mint '{token.mint}'.")
            else:
                report.add_detail(f"Token account mint '{token.mint}' balance: {token_account} {token.name}")

        # Should have an open orders account for each market in the group. (Only required for re-balancing via Serum, which isn't implemented here yet.)
        for market in group.markets:
            open_orders = OpenOrders.load_for_market_and_owner(context, market.address, account_address, context.dex_program_id, market.base.decimals, market.quote.decimals)
            if len(open_orders) == 0:
                report.add_warning(f"No Serum open orders account for market '{market.base.name}/{market.quote.name}' [{market.address}]'.")
            else:
                for open_orders_account in open_orders:
                    report.add_detail(f"Serum open orders account for market '{market.base.name}/{market.quote.name}': {open_orders_account}")

        # May have one or more Mango Markets margin account, but it's optional for liquidating
        margin_accounts = MarginAccount.load_all_for_owner(context, account_address, group)
        if len(margin_accounts) == 0:
            report.add_detail(f"Account '{account_address}' has no Mango Markets margin accounts.")
        else:
            for margin_account in margin_accounts:
                report.add_detail(f"Margin account: {margin_account}")

        return report

    # It would be good to be able to fix an account automatically, which should
    # be possible if a Wallet is passed.
    def prepare_wallet_for_group(self, wallet: Wallet, group: Group) -> ScoutReport:
        report = ScoutReport(wallet.address)
        report.add_error("AccountScout can't yet prepare wallets.")
        return report


# 🏃 Running

You can run the following cell to check any user account to make sure it has the proper sub-accounts set up and available.

Enter the public key of the account you want to verify in the value for `ACCOUNT_TO_VERIFY` in the box below, between the double-quote marks. Then run the notebook by choosing 'Run > Run All Cells' from the notebook menu at the top of the page.

In [ ]:
ACCOUNT_TO_VERIFY = ""

In [ ]:
if __name__ == "__main__":
    logging.getLogger().setLevel(logging.INFO)

    if ACCOUNT_TO_VERIFY == "":
        raise Exception("No account to look up - try setting the variable ACCOUNT_TO_LOOK_UP to an account public key.")

    from Context import default_context

    print("Context:", default_context);

    root_account_key = PublicKey(ACCOUNT_TO_VERIFY)
    group = Group.load(default_context)

    scout = AccountScout()
    report = scout.verify_account_prepared_for_group(default_context, group, root_account_key)

    print(report)
